# Inspect Runnables

- Author: [ranian963](https://github.com/ranian963)
- Peer Review: []()
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/13-LangChain-Expression-Language/02-InspectRunnables.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/13-LangChain-Expression-Language/02-InspectRunnables.ipynb)

## Overview
In this tutorial, we will explore how to **inspect** and visualize different components (including the graph structure) of a `Runnable` chain. Understanding the underlying graph structure helps you diagnose and optimize complex chain workflows.

### Table of Contents
- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Introduction to Inspecting Runnables](#introduction-to-inspecting-runnables)
  - [Graph Inspection](#graph-inspection)
  - [Graph Output](#graph-output)
  - [Prompt Retrieval](#prompt-retrieval)

  ### References
- [LangChain: Runnables](https://python.langchain.com/api_reference/core/runnables.html)
- [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)
----


## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
        "faiss-cpu",
        "grandalf",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "02-InspectRunnables",
    }
)

You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## Introduction to Inspecting Runnables

LangChain `Runnable` objects can be combined into pipelines, which we call **chains** or **flows**. After creating a `Runnable`, you may want to **examine its structure** to understand what's happening internally.

By inspecting these components, you can:
- Understand how data transforms and flows through the pipeline
- View the graph structure for debugging purposes
- Access or modify prompts and sub-chains as needed

### Graph Inspection 

We'll build a runnable chain consisting of a FAISS retriever, a prompt template, and a ChatOpenAI model. Then we'll **examine the chain's graph structure** to understand the data flow between these components.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Create a FAISS vector store with sample text data
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)

# Initialize a retriever from the vector store
retriever = vectorstore.as_retriever()

template = """Answer the question using only the following context:\n{context}\n\nQuestion: {question}"""
# Set up the prompt template
prompt = ChatPromptTemplate.from_template(template)

# Initialize the ChatOpenAI model
model = ChatOpenAI(model="gpt-4-mini")

# Build the chain: (dictionary input) => prompt => model => output parser
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }  # Get context and question
    | prompt
    | model
    | StrOutputParser()
)

### Graph Output
We can **inspect** the chain’s internal graph of nodes (steps) and edges (data flows).

In [ ]:
# Get nodes from the chain's graph
chain.get_graph().nodes

In [ ]:
# Get edges from the chain's graph
chain.get_graph().edges

We can also print the graph in an ASCII-based diagram to visualize the chain flow.

In [ ]:
chain.get_graph().print_ascii()

### Prompt Retrieval
Finally, we can retrieve the **actual prompts** used in this chain. This is helpful to see exactly what LLM instructions are being sent.


In [ ]:
chain.get_prompts()